## Setup

### Imports

In [ ]:
import os
from pathlib import Path

from dotenv import load_dotenv
from loguru import logger
from mistralai import Mistral

import pandas as pd
from langchain_core.documents import Document
from langchain_text_splitters import (MarkdownHeaderTextSplitter,
                                      RecursiveCharacterTextSplitter)

### Config

In [ ]:
load_dotenv()

mistral_api_key = os.getenv("MISTRAL_API_KEY")
if not mistral_api_key:
    raise ValueError("Mistral api key not present in .env")

mistral_client = Mistral(api_key=mistral_api_key)

party = "50PLUS"

markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=[
        ("#", "Hoofdstuk"),
        ("##", "Sectie"),
        ("###", "Subsectie"),
    ],
    strip_headers=True,
)

recursive_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=50,
    separators=["\n\n", ".\n", "\n", ".", " ", ""],
    keep_separator="end",
)

## Process PDF with Mistral OCR

### Upload PDF to Mistral

In [ ]:
pdf_filename = f"Verkiezingsprogramma {party}.pdf"
pdf_filepath = Path.cwd().parent / "data" / "pdfs" / pdf_filename

if not pdf_filepath.exists():
    raise ValueError(f"The file {pdf_filepath} does not exist.")

logger.info(f"Uploading {pdf_filename} to Mistral...")
uploaded_pdf = mistral_client.files.upload(
    file={
        "file_name": pdf_filename,
        "content": open(pdf_filepath, "rb"),
    },
    purpose="ocr"
)
document_url = mistral_client.files.get_signed_url(file_id=uploaded_pdf.id)

### Process uploaded document

In [ ]:
logger.info(f"Running OCR on document {document_url}...")
ocr_result = mistral_client.ocr.process(
    model="mistral-ocr-latest",
    document={
        "type": "document_url",
        "document_url": document_url.url,
    },
    include_image_base64=False
)

### Extract markdown from results

In [ ]:
response_markdown = '\n\n'.join([page.markdown for page in ocr_result.pages])
print(response_markdown)

## Markdown Cleanup (do manually)
The OCR response is pretty accurate, but can still make errors which result in typos or formatting errors.
Especially markdown headers are often not structured correctly, so we need to correct it manually.
Below, we load the "cleaned" and shortened version of the markdown file; this only contains the main chapters, without introductions or summaries.

## Chunking

### Split markdown file

In [ ]:
short_markdown_file = Path.cwd().parent / "data" / "markdown_short" / f"{party}_short.md"

with open(short_markdown_file, 'r', encoding='utf-8') as file:
    markdown_string = file.read()

# Step 1: Split the markdown text by headers
md_header_splits = markdown_splitter.split_text(markdown_string)

# Step 2: Recursively split the header chunks into smaller chunks
chunks = recursive_splitter.split_documents(md_header_splits)

example_chunk = chunks[20]
example_chunk

### Chunks visualization

In [ ]:
def chunks_to_dataframe(chunks: list[Document]) -> pd.DataFrame:
    """
    Converts a list of LangChain Document objects into a pandas DataFrame,
    extracting specified metadata fields.
    """
    data = []
    
    for chunk in chunks:
        metadata = chunk.metadata
        page_content = chunk.page_content

        # Extract metadata fields with a default value of None or an empty string
        hoofdstuk = metadata.get('Hoofdstuk', "")
        sectie = metadata.get('Sectie', "")
        subsectie = metadata.get('Subsectie', "")

        data.append({
            'Partij': party,
            'Hoofdstuk': hoofdstuk,
            'Sectie': sectie,
            'Subsectie': subsectie,
            'Text': page_content,
        })
        
    return pd.DataFrame(data)

In [ ]:
vector_db = chunks_to_dataframe(chunks)[:50]
vector_db

## Embedding

### Process document chunks for embedding

In [ ]:
def format_embedding_content(chunk: Document) -> str:
    metadata = chunk.metadata
    page_content = chunk.page_content

    chapter_title = metadata.get("Hoofdstuk")
    if not chapter_title:
        raise ValueError("No chapter title found")

    section_title = metadata.get("Sectie", "")
    subsection_title = metadata.get("Subsectie", "")

    embedding_content = f"{chapter_title}\n{section_title}\n{subsection_title}\n{page_content}"

    return embedding_content

In [ ]:
embedding_content = format_embedding_content(example_chunk)

print(embedding_content)

In [ ]:
vector_db["embedding_content"] = vector_db["Hoofdstuk"] + '\n' + vector_db["Sectie"] + '\n' + vector_db["Subsectie"] + '\n' + vector_db["Text"]
vector_db

### Embed content

In [ ]:
from google import genai
from google.genai import types


client = genai.Client()

embedding_result = client.models.embed_content(
        model="gemini-embedding-001",
        contents=list(vector_db["embedding_content"])
        )

In [ ]:
embedding_list = [item.values for item in embedding_result.embeddings]
vector_db['embeddings'] = embedding_list
display(vector_db)

### Process db query
TODO: improve logic and add threshold filtering

In [ ]:
import numpy as np

def find_best_passage(query, dataframe, num_results: int = 3, threshold: float = 0):
  """
  Compute the distances between the query and each document in the dataframe
  using the dot product.
  """
  query_embedding = client.models.embed_content(
      model="gemini-embedding-001",
      contents=query,
      config=types.EmbedContentConfig(
          task_type="retrieval_query",
          )
  )

  dot_products = np.dot(
      np.stack(dataframe['embeddings']),
      query_embedding.embeddings[0].values
  )
  dataframe["dot_product"] = dot_products
  index_ranking = np.argsort(dot_products)
  top_indices = index_ranking[-num_results:][::-1]
  return dataframe.iloc[top_indices] # Return text from index with max value

In [ ]:
query = "Wat vind de partij van kunstmatige intelligentie?"

top_df = find_best_passage(query, vector_db, num_results=3, threshold=0.6)
display(top_df)

In [ ]:
for i, result in top_df.iterrows():
    display(result["embedding_content"])

## Question answering
TODO: clean up, improve prompt, define format_return_text

In [ ]:
import textwrap

def make_prompt(query, relevant_passages):
  escaped = (
      relevant_passages
      .replace("'", "")
      .replace('"', "")
      .replace("\n", " ")
  )
  prompt = textwrap.dedent("""
    You are a helpful and informative bot that answers questions using text
    from the reference passage included below. Be sure to respond in a
    complete sentence, being comprehensive, including all relevant
    background information.

    However, you are talking to a non-technical audience, so be sure to
    break down complicated concepts and strike a friendly and conversational
    tone. If the passage is irrelevant to the answer, you may ignore it.

    QUESTION: '{query}'
                           
    PASSAGES: '{relevant_passage}'

    ANSWER:
  """).format(query=query, relevant_passage=escaped)


  return prompt

In [ ]:
def format_return_text(result_df: pd.DataFrame) -> list[str]:
    """Combines Header titles and text into a readable format for each entry"""
    # TODO
    return

In [ ]:
retrieved_passages = format_return_text(top_df)

prompt = make_prompt(query, retrieved_passages)

In [ ]:
answer = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=prompt,
)
print(answer.text)